In [31]:
from block import Block
from transaction import Transaction
from typing import List
import time
import json


class BlockChain:

    def __init__(self) -> None:
        self.transactions: List[Transaction] = []
        self.chain: List[Block] = []
        self.difficulty = 5
        # Append Gensis Block
        self.chain.append(Block.create_gensis_block())

    def proof_of_work(self, block: Block) -> Block:
        t1 = time.time()
        while True:
            if block.compute_hash()[:self.difficulty] == "0" * self.difficulty:
                t2 = time.time()
                print(f"Proof of Work: {t2 - t1:.2f}s")
                return block
            block.nonce += 1

    def validate_block(self, block: Block) -> bool:
        last_block = self.chain[-1]

        if block.previous_hash != last_block.compute_hash():
            print("Hash conflict!")
            return False

        if block.compute_hash()[:self.difficulty] != "0" * self.difficulty:
            print("Hash Incorrect!")
            return False

        self.chain.append(block)
        print("Block added successfully to chain")
        return True

    def add_transaction(self, transactions) -> None:
        self.transactions.append(transactions)

    def mine(self) -> bool:
        if len(self.transactions) < 1:
            print("No transaction/s found")
            return False

        last_block = self.chain[-1]
        block = Block(transactions=self.transactions,
                      previous_hash=last_block.compute_hash())

        block = self.proof_of_work(block)

        if not self.validate_block(block):
            print("Error Block NOT Valid")
            return False

        self.transactions = []
        return True

    def to_json(self) -> str:
        out = []
        for index, block in enumerate(self.chain):
            print(block)
            temp = {"hash": block.compute_hash(),
                    "height": index}
            temp.update(block.__dict__)
            out.append(temp)

        return json.dumps(out)


def main():
    bitcoins = BlockChain()

    t1 = Transaction(from_address="1234",
                     to_address="9876",
                     amount=0.123)

    t2 = Transaction(from_address="5456",
                     to_address="85643",
                     amount=34562.21)

    t3 = Transaction(from_address="1234",
                     to_address="9393",
                     amount=10.3)

    print(bitcoins.to_json())

    bitcoins.add_transaction([t1, t2, t3])
    bitcoins.mine()

    print(bitcoins.to_json())

In [32]:
main()

Block(version=1, timestamp=1638956211, difficulty=2, nonce=0, transactions=[])
[{"hash": "c45222fe25ff92e063d78b36b4036038e0e12bec3325516c158ba91d20ced0be", "height": 0, "version": "1", "previous_hash": "0", "timestamp": 1638956211, "difficulty": 2, "nonce": 0, "transactions": []}]
Proof of Work: 6.51s
Block added successfully to chain
Block(version=1, timestamp=1638956211, difficulty=2, nonce=0, transactions=[])
Block(version=1, timestamp=1638956211, difficulty=2, nonce=1305386, transactions=[[Transaction(version_no=1, from_address=1234, to_address=9876), Transaction(version_no=1, from_address=5456, to_address=85643), Transaction(version_no=1, from_address=1234, to_address=9393)]])


TypeError: Object of type Transaction is not JSON serializable

In [8]:
hashlib.sha256(str(b.block_header).encode()).hexdigest()

'9520ad264e4de8694be978ea444f8a8f6157592483fb18f16439eac0b170914a'